In [1]:
import pandas as pd
import numpy as np

In [2]:
ana = pd.read_excel("../dat/validation/deps_to_validate_ana.xlsx")
hanqi = pd.read_excel("../dat/validation/deps_to_validate_hanqi.xlsx")
dom = pd.read_excel("../dat/validation/deps_to_validate_Dom.xlsx")
atc = pd.read_excel("../dat/validation/deps-all-validated-ATC.xlsx")

In [3]:
atc = atc.sort_values(atc.columns[0]).reset_index()

In [4]:
source = pd.read_csv("../dat/validation/source_column.csv")

In [23]:
ratings = pd.DataFrame({"source": source["source"],
                        "rater1": ana.iloc[:,3],
                        "rater2": hanqi.iloc[:,3],
                        "rater3": dom.iloc[:,3],
                        "rater4": atc.iloc[:,4]}).replace(-1, 0).replace("D", 0).replace("?", 0)

In [6]:
def kappa(ratings1, ratings2, chance=0.25):
    agree = 1-np.mean((ratings1-ratings2)**2)
    return round((agree-chance)/(1-chance),2)

print([kappa(ratings.iloc[:,1], ratings.iloc[:,2]),
kappa(ratings.iloc[:,1], ratings.iloc[:,3]),
kappa(ratings.iloc[:,2], ratings.iloc[:,3]),
kappa(ratings.iloc[:,1], ratings.iloc[:,4]),
kappa(ratings.iloc[:,2], ratings.iloc[:,4]),
kappa(ratings.iloc[:,3], ratings.iloc[:,4]),])

[0.6, 0.75, 0.68, 0.7, 0.67, 0.76]


In [7]:
ratings.groupby("source", as_index=False).mean()#.iloc[:,1:].mean(axis=1)

,source,rater1,rater2,rater3,rater4
0,llm,0.33,0.64,0.48,0.420000
1,textbook,0.33,0.45,0.37,0.330000
2,wiki,0.63,0.73,0.76,0.818182


In [36]:
import plotly.express as px
N=4
kappa_coefs = [[kappa(ratings.iloc[:,i+1], ratings.iloc[:,j+1]) for j in range(N) ] for i in range (N)]
fig = px.imshow(kappa_coefs, text_auto=True, labels=dict(color="Kappa coefficient"),
                x=['Rater 1', 'Rater 2', 'Rater 3', 'Rater 4'],
                y=['Rater 1', 'Rater 2', 'Rater 3', 'Rater 4'],
                color_continuous_scale='Greys')
fig.update_layout({
"plot_bgcolor": "rgba(0, 0, 0, 0)",
"paper_bgcolor": "rgba(0, 0, 0, 0)",
})
fig.update_coloraxes(showscale=False)
fig.show()